# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 26 2022 2:35PM,247307,20,8518561,"ACI Healthcare USA, Inc.",Released
1,Sep 26 2022 2:34PM,247306,10,0086006504,ISDIN Corporation,Released
2,Sep 26 2022 2:34PM,247306,10,0086006503,ISDIN Corporation,Released
3,Sep 26 2022 2:34PM,247306,10,0086006546,ISDIN Corporation,Released
4,Sep 26 2022 2:34PM,247306,10,0086006547,ISDIN Corporation,Released
5,Sep 26 2022 2:34PM,247306,10,0086006550,ISDIN Corporation,Released
6,Sep 26 2022 2:34PM,247306,10,0086006548,ISDIN Corporation,Released
7,Sep 26 2022 2:34PM,247306,10,0086006549,ISDIN Corporation,Released
8,Sep 26 2022 2:34PM,247306,10,0086006557,ISDIN Corporation,Released
9,Sep 26 2022 2:34PM,247306,10,0086006558,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,247303,Released,17
39,247304,Released,1
40,247305,Released,3
41,247306,Released,24
42,247307,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247303,NaN,NaN,17.0
247304,NaN,NaN,1.0
247305,NaN,NaN,3.0
247306,NaN,NaN,24.0
247307,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247220,0.0,0.0,3.0
247223,0.0,0.0,8.0
247238,0.0,0.0,1.0
247240,0.0,1.0,0.0
247246,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247220,0,0,3
247223,0,0,8
247238,0,0,1
247240,0,1,0
247246,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247220,0,0,3
1,247223,0,0,8
2,247238,0,0,1
3,247240,0,1,0
4,247246,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247220,,,3
1,247223,,,8
2,247238,,,1
3,247240,,1,
4,247246,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 26 2022 2:35PM,247307,20,"ACI Healthcare USA, Inc."
1,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation
25,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation
28,Sep 26 2022 2:24PM,247304,10,Yusen – 3D Matrix
29,Sep 26 2022 2:22PM,247303,10,"Snap Medical Industries, LLC"
46,Sep 26 2022 2:21PM,247302,10,Eywa Pharma Inc.
47,Sep 26 2022 2:17PM,247301,10,"Methapharm, Inc."
48,Sep 26 2022 2:17PM,247300,20,"Exact-Rx, Inc."
58,Sep 26 2022 2:15PM,247299,10,Emerginnova
59,Sep 26 2022 2:13PM,247298,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 26 2022 2:35PM,247307,20,"ACI Healthcare USA, Inc.",,,1
1,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation,,,24
2,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation,,,3
3,Sep 26 2022 2:24PM,247304,10,Yusen – 3D Matrix,,,1
4,Sep 26 2022 2:22PM,247303,10,"Snap Medical Industries, LLC",,,17
5,Sep 26 2022 2:21PM,247302,10,Eywa Pharma Inc.,,,1
6,Sep 26 2022 2:17PM,247301,10,"Methapharm, Inc.",,,1
7,Sep 26 2022 2:17PM,247300,20,"Exact-Rx, Inc.",,,10
8,Sep 26 2022 2:15PM,247299,10,Emerginnova,,,1
9,Sep 26 2022 2:13PM,247298,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 2:35PM,247307,20,"ACI Healthcare USA, Inc.",1,,
1,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation,24,,
2,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation,3,,
3,Sep 26 2022 2:24PM,247304,10,Yusen – 3D Matrix,1,,
4,Sep 26 2022 2:22PM,247303,10,"Snap Medical Industries, LLC",17,,
5,Sep 26 2022 2:21PM,247302,10,Eywa Pharma Inc.,1,,
6,Sep 26 2022 2:17PM,247301,10,"Methapharm, Inc.",1,,
7,Sep 26 2022 2:17PM,247300,20,"Exact-Rx, Inc.",10,,
8,Sep 26 2022 2:15PM,247299,10,Emerginnova,1,,
9,Sep 26 2022 2:13PM,247298,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 2:35PM,247307,20,"ACI Healthcare USA, Inc.",1,,
1,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation,24,,
2,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation,3,,
3,Sep 26 2022 2:24PM,247304,10,Yusen – 3D Matrix,1,,
4,Sep 26 2022 2:22PM,247303,10,"Snap Medical Industries, LLC",17,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 2:35PM,247307,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
1,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation,24.0,NaN,NaN
2,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation,3.0,NaN,NaN
3,Sep 26 2022 2:24PM,247304,10,Yusen – 3D Matrix,1.0,NaN,NaN
4,Sep 26 2022 2:22PM,247303,10,"Snap Medical Industries, LLC",17.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 2:35PM,247307,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
1,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation,24.0,0.0,0.0
2,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation,3.0,0.0,0.0
3,Sep 26 2022 2:24PM,247304,10,Yusen – 3D Matrix,1.0,0.0,0.0
4,Sep 26 2022 2:22PM,247303,10,"Snap Medical Industries, LLC",17.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6429211,91.0,1.0,1.0
12,247271,0.0,1.0,0.0
16,741812,5.0,0.0,0.0
19,989131,50.0,0.0,0.0
20,989162,47.0,12.0,4.0
21,247246,0.0,1.0,0.0
22,247298,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6429211,91.0,1.0,1.0
1,12,247271,0.0,1.0,0.0
2,16,741812,5.0,0.0,0.0
3,19,989131,50.0,0.0,0.0
4,20,989162,47.0,12.0,4.0
5,21,247246,0.0,1.0,0.0
6,22,247298,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,91.0,1.0,1.0
1,12,0.0,1.0,0.0
2,16,5.0,0.0,0.0
3,19,50.0,0.0,0.0
4,20,47.0,12.0,4.0
5,21,0.0,1.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,91.0
1,12,Released,0.0
2,16,Released,5.0
3,19,Released,50.0
4,20,Released,47.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21,22
Status,,,,,,,
Completed,1.0,0.0,0.0,0.0,4.0,0.0,0.0
Executing,1.0,1.0,0.0,0.0,12.0,1.0,0.0
Released,91.0,0.0,5.0,50.0,47.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,4.0,0.0,0.0
1,Executing,1.0,1.0,0.0,0.0,12.0,1.0,0.0
2,Released,91.0,0.0,5.0,50.0,47.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,4.0,0.0,0.0
1,Executing,1.0,1.0,0.0,0.0,12.0,1.0,0.0
2,Released,91.0,0.0,5.0,50.0,47.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()